In [1]:
!pip3 install --upgrade google-cloud-documentai
!pip3 install --upgrade google-cloud-storage
!pip3 install --upgrade google-cloud-documentai-toolbox

Defaulting to user installation because normal site-packages is not writeable


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Defaulting to user installation because normal site-packages is not writeable


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Defaulting to user installation because normal site-packages is not writeable


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


In [37]:
from google.api_core.client_options import ClientOptions
from google.cloud import documentai

PROJECT_ID = "tally-ai-394207"
LOCATION = "us"  # Format is 'us' or 'eu'
PROCESSOR_ID = "d5efc50f9221c006"  # Create processor in Cloud Console

FILE_PATH = "C:/Users/manish.g/Downloads/DocumentAI (2)/DocumentAI/InvoiceImages/79.jpeg"

#local file in CWD
# Refer to https://cloud.google.com/document-ai/docs/file-types for supported file types (MIME)

#MIME_TYPE = ""

MIME_TYPE = "image/jpeg"

field_mask= "text"

# Instantiates a client
docai_client = documentai.DocumentProcessorServiceClient(
    client_options=ClientOptions(api_endpoint=f"{LOCATION}-documentai.googleapis.com")
)

# The full resource name of the processor

RESOURCE_NAME = docai_client.processor_path(PROJECT_ID, LOCATION, PROCESSOR_ID)

# Read the file into memory
with open(FILE_PATH, "rb") as image:
    image_content = image.read()

# Load Binary Data into Document AI RawDocument Object
raw_document = documentai.RawDocument(content=image_content, mime_type=MIME_TYPE)

# Configure the process request
request = documentai.ProcessRequest(name=RESOURCE_NAME, raw_document=raw_document)

# Use the Document AI client to process the sample form
result = docai_client.process_document(request=request)

document_object = result.document
print("Document processing complete. Below is the output.")

# print(f"{document_object.text}") #need this
# print(f"Number of pages in the pdf: {len(document_object.pages)}") #don't need this

def layout_to_text(layout: documentai.Document.Page.Layout, text: str) -> str:
    """
    Document AI identifies text in different parts of the document by their
    offsets in the entirety of the document"s text. This function converts
    offsets to a string.
    """
    # If a text segment spans several lines, it will
    # be stored in different text segments.
    return "".join(
        text[int(segment.start_index) : int(segment.end_index)]
        for segment in layout.text_anchor.text_segments
    )



Document processing complete. Below is the output.


In [38]:
def draw_bounding_box( normalized_vertices,entity_type, color=(0, 255, 0), thickness=2):
    height, width, _ = image.shape

    # Convert normalized vertices to pixel coordinates
    x_min, y_min = (
        int(min(normalized_vertex.x for normalized_vertex in normalized_vertices) * width),
        int(min(normalized_vertex.y for normalized_vertex in normalized_vertices) * height)
    )

    x_max, y_max = (
        int(max(normalized_vertex.x for normalized_vertex in normalized_vertices) * width),
        int(max(normalized_vertex.y for normalized_vertex in normalized_vertices) * height)
    )

    # Draw the bounding box as a rectangle
    cv2.rectangle(image, (x_min, y_min), (x_max, y_max), color=color, thickness=thickness)
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 1
    font_thickness = 2
    text_size = cv2.getTextSize(entity_type, font, font_scale, font_thickness)[0]
    text_color = (255,0,0)

    text_position = (x_min, max(y_min - 5, 0))  # Adjust the vertical position as needed
    cv2.putText(image, entity_type, text_position, font, font_scale, text_color, font_thickness)

In [39]:
import cv2

# ... (your existing code)
# Save the image with bounding boxes
def save_image(input):
    filename=FILE_PATH.split('/')[-1]
    filename
    output_image_path = f"C:/Users/manish.g/Downloads/DocumentAI (2)/DocumentAI/InvoiceImages/{input}_{filename}" 
    # Replace with your desired path
    cv2.imwrite(output_image_path, image)
    
    # Display the saved image path
    print(f"Image with bounding boxes saved at: {output_image_path}")

In [40]:
entities = document_object.entities
entities

[text_anchor {
  text_segments {
    start_index: 1830
    end_index: 1839
  }
  text_segments {
    start_index: 1840
    end_index: 1842
  }
  text_segments {
    start_index: 1844
    end_index: 1853
  }
  text_segments {
    start_index: 1854
    end_index: 1856
  }
  text_segments {
    start_index: 1858
    end_index: 1867
  }
  text_segments {
    start_index: 1868
    end_index: 1877
  }
}
type_: "vat"
mention_text: "83,228.00 14 11,651.92 14 11,651.92 23,303.84"
confidence: 1
page_anchor {
  page_refs {
    bounding_poly {
      normalized_vertices {
        x: 0.419758052
        y: 0.812945545
      }
      normalized_vertices {
        x: 0.857258081
        y: 0.812945545
      }
      normalized_vertices {
        x: 0.857258081
        y: 0.825491846
      }
      normalized_vertices {
        x: 0.419758052
        y: 0.825491846
      }
    }
  }
}
id: "0"
properties {
  text_anchor {
    text_segments {
      start_index: 1830
      end_index: 1839
    }
    content: 

In [41]:
c=1
import cv2
image_path = FILE_PATH
image = cv2.imread(image_path)
image.shape

for item in entities:
    print(c)
    print(item.type_," :",item.mention_text )
    if "page_anchor" in item:
        if "page_refs" in item.page_anchor:
            
            for page_ref in item.page_anchor.page_refs:
                if "normalized_vertices" in page_ref.bounding_poly:
                    draw_bounding_box(page_ref.bounding_poly.normalized_vertices,item.type_)  
    if "properties" in item:
        for prop_item in item.properties:
            if "page_anchor" in prop_item:
                if "page_refs" in prop_item.page_anchor:
                    
                    for prop_page_ref in prop_item.page_anchor.page_refs:
                        if "normalized_vertices" in prop_page_ref.bounding_poly:
                            prop_type=str(prop_item.type_).split('/')
                            draw_bounding_box(prop_page_ref.bounding_poly.normalized_vertices,prop_type[1],thickness=1)
                
    c=c+1

save_image("InvoiceParser_Entities")

1
vat  : 83,228.00 14 11,651.92 14 11,651.92 23,303.84
2
invoice_date  : 19-12-2022
3
invoice_id  : KR2223/00600
4
total_tax_amount  : 23,303.84
5
supplier_name  : Pentagon System And Services Pvt. Ltd.
6
supplier_address  : 444, First Floor, 8th Main, 4th Block, Koramangala,
Bangalore Karnataka - 560034Tel. No. 080 2563 5220 Fax
7
supplier_email  : pentagonbge@pentagon.co.in
8
total_amount  : 1,06,531.84
9
net_amount  : 83,228.00
10
receiver_name  : TALLY SOLUTIONS PVT LTD
11
supplier_tax_id  : 29AAACP0547J1ZC
12
receiver_tax_id  : AAACP7879D
13
ship_to_name  : TALLY SOLUTIONS PVT LTD
14
invoice_type  : 
15
purchase_order  : PO-HO/0428/22
16
ship_to_address  : AMR Tech Park II, No 23 & 24,
17
line_item  : Dell 34 Monitor - C3422WE 1.00 Nos 83,228.00 11,651.92
18
line_item  : Boxing Label Mod
6Amp Power Cord (India only)
Y4-5 Premium Panel with Advanced Exchange Service Extension
3Y Premium Panel with Advanced Exchange Service
Sr No C9HZYN3
Image with bounding boxes saved at: C:/Users/

In [42]:
fields=document_object.pages[0].form_fields 

In [43]:
fields

[]

In [44]:
import cv2
image_path = FILE_PATH
image = cv2.imread(image_path)
image.shape


count=1
for field in fields:
    print(count)
    if "field_name" in field and "text_anchor" in field.field_name and "content" in field.field_name.text_anchor:
        print(field.field_name.text_anchor.content)
        draw_bounding_box(field.field_name.bounding_poly.normalized_vertices,str(count))   
    
    if "field_value" in field and "text_anchor" in field.field_value and "content" in field.field_value.text_anchor:
        print(field.field_value.text_anchor.content)
        draw_bounding_box(field.field_value.bounding_poly.normalized_vertices,str(count))   
        
    count = count+1

save_image("InvoiceParser_FormField")

Image with bounding boxes saved at: C:/Users/manish.g/Downloads/DocumentAI (2)/DocumentAI/InvoiceImages/InvoiceParser_FormField_79.jpeg
